In [1]:
from db_import import *
from datetime import datetime, timedelta
import numpy as np
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', 30)
pd.options.display.float_format = '{:.2f}'.format
from forex_python.converter import CurrencyRates

In [2]:
# Data Import
end_date = datetime.now().strftime('%Y-%m-%d')
query_obj = Queries('2022-01-01', end_date)
db_obj = DBImport(db_type='cscart')

analytics = db_obj.data_import(query_obj.analytics_query)
brand = db_obj.data_import(query_obj.brand_query)
inventory = db_obj.data_import(query_obj.inventory_query)
category = db_obj.data_import(query_obj.category_query)
mainexposure = db_obj.data_import(query_obj.mainexposure_query)

In [3]:
# basic preprocessing
analytics['purchased_ymd'] = pd.to_datetime(analytics.purchased_at).dt.normalize() # 시간 제외한 날짜만
analytics =  analytics[analytics['company_name']!='몬스터짐 이벤트'] # 대회류 아닌 케이스만
brand['brand'] = brand['brand'].apply(lambda x: x.strip())

# get exchange rate
exchange_rate = CurrencyRates()
usd_krw = exchange_rate.get_rates('USD')['KRW']

# 환율 고려하여 원단위 매출 연산
analytics['krw_price'] = analytics.apply(lambda x: x['marked_down_price'] * usd_krw if x['currency']=='USD' else x['marked_down_price'], axis=1)
analytics['krw_sales'] = analytics['krw_price'] * analytics['product_qty']

In [4]:
# product-barcode matching table
prod_bar_match = analytics[['product_id', 'barcode']].drop_duplicates()

# barcode-product_name matching table (추후 바코드 없는 상품들은 바코드 기준 조인이 안되기 때문에 상품명, 옵션명 테이블 따로 생성)
bar_prod_name = analytics[['product_id', 'product_name_kor']].drop_duplicates()
bar_prod_name = bar_prod_name.groupby('product_id')['product_name_kor'].sum().reset_index() # 한 상품명 내에서 여러 옵션이 담겨있기도 함. 모든 옵션을 보여주기 위해 합치기

# barcode-variant_name matching table (추후 바코드 없는 상품들은 바코드 기준 조인이 안되기 때문에 상품명, 옵션명 테이블 따로 생성)
bar_var_name = analytics[['barcode', 'variant_1_name_kor']].drop_duplicates()
bar_var_name = bar_var_name.groupby('barcode')['variant_1_name_kor'].sum().reset_index() # 한 바코드 내에서 여러 옵션이 담겨있기도 함. 모든 옵션을 보여주기 위해 합치기

# product_id-vendor matching table
prod_vendor = analytics[['product_id', 'company_name']].drop_duplicates()

# 판매 관련 확인

- 판매 급상승의 경우, 과거 판매가 된 날짜만을 바탕으로 진행. 즉, 재고가 있지만 판매가 일어나지 않아 **판매량을 0으로 집계하는 과정은 생략 (∵ ① 지나친 저회전 상품에 대해 급상승을 보고싶은 것도 아니고, ② 재고 정보가 부정확할 수도 있음. 무엇보다도, ③ 0을 복구해줌으로서 분모에 위치시킬 경우 왜곡이 발생할 수 있기 때문)**
- 현재 날짜를 기준으로 최근 일주일을 **타겟기간**, 일주일전으로부터 14일 전까지를 **비교기간**으로 설정 → 각 기간 내에서도 OOS로 인해 판매가 일어나지 않았을 경우, 단순 합으로 비교가 어렵기 때문에 주단위 판매량 합이 아니라, **각 기간의 일평균**으로 연산
- 위와같이 설정할 경우, 비교기간동안 판매가 0건인 상품의 경우 inf로 연산되기 때문에 **비교기간동안 판매가 없는 상품**의 경우 별도 처리 필요 → **1로 대체** (∵ 최근 입고 등의 이슈로 최근 판매량이 급상승 한 상품은 주로 추적을 원하기 때문에, 판매수량이 그대로 입력되도록 분모를 1로 설정. 단, 판매량이 어느정도 되어야 재입고 급상승의 의미가 있기 때문에, **타겟기간 최근 판매량에 threshold 설정**)
- ~~상대적으로 중요도가 떨어지는, **판매량이 낮은 상품들**도 비율상으로 커져보일 수 있음. **비교기간 평균판매량에 threshold 설정**~~ → 제외. 바로 위에서 최근 재입고 상품의 경우 분모가 1이되어야 하는데, 비교기간 평균판매량에 threshold가 1보다 크게 걸리면 해당 상품들은 제외되기 때문
- 카테고리의 경우, 기존의 category_M, category_S 두 카테고리를 활용하는 것이 아니라, 상품:카테고리의 1:N 매칭이 이루어지도록 카테고리 테이블을 활용하여 한 상품이 여러 카테고리에 중복 집계가 가능하도록 연산
- 브랜드, 카테고리별 결과의 경우 상위 10개의 결과에 해당하는 상품별 상세정보 테이블 추가로 생성

In [5]:
# target and compare dates
target_end = datetime.now().strftime('%Y-%m-%d')
target_start = (datetime.now() - timedelta(days=7)).strftime('%Y-%m-%d')
compare_end = (datetime.now() - timedelta(days=8)).strftime('%Y-%m-%d')
compare_start = (datetime.now() - timedelta(days=15)).strftime('%Y-%m-%d')
target_compare_days = [compare_start, compare_end, target_start, target_end] # 원인확인 class 생성시 필요

In [6]:
### aggregation

## 1. 상품별 결과

# barcode 있는 상품들
daily_sales = analytics.groupby(['barcode', 'purchased_ymd'])['product_qty', 'krw_sales'].sum().reset_index()
daily_sales = daily_sales.fillna('-') # 결측값의 nan이 종류가 다양하여 '-'으로 통일
daily_sales = daily_sales[~(daily_sales['barcode'].isin(['-', '']))]
target_sales = daily_sales[(daily_sales['purchased_ymd']>target_start)&(daily_sales['purchased_ymd']<=target_end)]
target_sales = target_sales.groupby('barcode').agg({'krw_sales': 'sum', 'product_qty':'sum', 'purchased_ymd': pd.Series.nunique})
target_sales = target_sales.rename(columns={'krw_sales': 'target_krw_sales', 'product_qty':'target_qty', 'purchased_ymd':'target_days'})
compare_sales = daily_sales[(daily_sales['purchased_ymd']>compare_start)&(daily_sales['purchased_ymd']<=compare_end)]
compare_sales = compare_sales.groupby('barcode').agg({'krw_sales': 'sum', 'product_qty':'sum', 'purchased_ymd': pd.Series.nunique})
compare_sales = compare_sales.rename(columns={'krw_sales': 'compare_krw_sales', 'product_qty':'compare_qty', 'purchased_ymd':'compare_days'})
prod_sales = pd.merge(target_sales, compare_sales, on='barcode', how='outer')
prod_sales = prod_sales.reset_index()
prod_sales = pd.merge(prod_sales, prod_bar_match, on='barcode', how='left')

# barcode 없는 국내 일부 상품들
non_barcode = analytics[(analytics['barcode']=='')|(analytics['barcode'].isnull())]
target_sales_non = non_barcode[(non_barcode['purchased_ymd']>target_start)&(non_barcode['purchased_ymd']<=target_end)]
target_sales_non = target_sales_non.groupby('product_id').agg({'krw_sales': 'sum', 'product_qty':'sum', 'purchased_ymd': pd.Series.nunique})
target_sales_non = target_sales_non.rename(columns={'krw_sales': 'target_krw_sales', 'product_qty':'target_qty', 'purchased_ymd':'target_days'})
compare_sales_non = non_barcode[(non_barcode['purchased_ymd']>compare_start)&(non_barcode['purchased_ymd']<=compare_end)]
compare_sales_non = compare_sales_non.groupby('product_id').agg({'krw_sales': 'sum', 'product_qty':'sum', 'purchased_ymd': pd.Series.nunique})
compare_sales_non = compare_sales_non.rename(columns={'krw_sales': 'compare_krw_sales', 'product_qty':'compare_qty', 'purchased_ymd':'compare_days'})
prod_sales_non = pd.merge(target_sales_non, compare_sales_non, on='product_id', how='outer')
prod_sales_non = prod_sales_non.reset_index()

prod_sales = pd.concat([prod_sales, prod_sales_non])

# growth_rate 성장률 계산을 위해서는 판매량과 판매일자를 1로 채워줘야 하는데, 실제 값은 0으로 해야하기 때문에 저장해두고 추후 복구
target_days = prod_sales['target_days'].fillna(0)
compare_qty = prod_sales['compare_qty'].fillna(0)
compare_days = prod_sales['compare_days'].fillna(0)
prod_sales['target_qty'] = prod_sales['target_qty'].fillna(0) # target_qty는 0 그대로여도 상관없음. 항상 최우선 분자에 있기 때문. 나머지는 1로 채우기
prod_sales['target_days'] = prod_sales['target_days'].fillna(1) 
prod_sales['compare_qty'] = prod_sales['compare_qty'].fillna(1)
prod_sales['compare_days'] = prod_sales['compare_days'].fillna(1)
# prod_sales['growth_rate_daily'] = (prod_sales['target_qty']/prod_sales['target_days'] - prod_sales['compare_qty']/prod_sales['compare_days']) / (prod_sales['compare_qty']/prod_sales['compare_days']) # target과 compare의 날짜 수에 차이가 있을 수 있기 때문에, 일평균 판매량을 구한 후, 그 차이를 비교기간 대비로 연산
prod_sales['growth_rate_daily'] = (prod_sales['target_qty'] - prod_sales['compare_qty']) / prod_sales['compare_qty'] # 위처럼 일단위로 연산하니, 주간판매수량합산차이와 반대방향으로 연산되는 경우가 있어서 변경
prod_sales['target_days'] = target_days
prod_sales['compare_qty'] = compare_qty
prod_sales['compare_days'] = compare_days

# 매출액/판매량 차이 연산
prod_sales['target_krw_sales'] = prod_sales['target_krw_sales'].fillna(0)
prod_sales['compare_krw_sales'] = prod_sales['compare_krw_sales'].fillna(0)
prod_sales['krw_sales_diff'] = prod_sales['target_krw_sales'] - prod_sales['compare_krw_sales']
prod_sales['qty_diff'] = prod_sales['target_qty'] - prod_sales['compare_qty']

# 후처리
prod_sales['compare_qty'] = prod_sales['compare_qty'].astype('int')
prod_sales['compare_days'] = prod_sales['compare_days'].astype('int')
prod_sales['growth_rate_daily'] = np.round(prod_sales['growth_rate_daily'], 4) * 100
prod_sales['krw_sales_diff'] = np.round(prod_sales['krw_sales_diff']/1000) # 천단위 환산
prod_sales = pd.merge(prod_sales, brand, on='product_id', how='left')
prod_sales = pd.merge(prod_sales, bar_prod_name, on='product_id', how='left') # barcode 없는 상품들로 인해 상품명과 옵션명 별도로 조인
prod_sales = pd.merge(prod_sales, bar_var_name, on='barcode', how='left')
prod_sales = prod_sales.fillna('-')

In [7]:
class CauseTracker():
    def __init__(self, mainexposure, inventory, target_compare_days):
        self.mainexposure = mainexposure
        self.inventory = inventory
        self.target_compare_days = target_compare_days
        self.col_name_dict =  {'barcode': '바코드', # 추후 컬럼명 한국어로 변경 시 공통 활용되는 dictionary
                               'target_qty': '최근일주일판매량',
                               'target_days': '최근일주일판매일수',
                               'compare_qty': '과거일주일판매량',
                               'compare_days': '과거일주일판매일수',
                               'growth_rate_daily': '판매량증감률(%)',
                               'product_id': '상품번호',
                               'brand': '브랜드',
                               'product_name_kor': '상품명',
                               'variant_1_name_kor': '옵션명',
                               'target_category': '최근일주일노출영역',
                               'compare_category': '과거일주일노출영역',
                               'exposure_comment': '원인_메인노출여부',
                               'target_amount': '현재고',
                               'compare_amount': '과거일주일재고(MAX)',
                               'amt_gap': '재고변동',
                               'inventory_comment': '원인_재고여부',
                               'category': '카테고리',
                               'company_name': '벤더사',
                               'krw_sales_diff': '매출증감액(천원)',
                               'qty_diff': '판매량증감수량(개)',
                               'rank': '순위'
                              }

    def comment_exposure(self, arr):
        if (arr['target_category']=='-')&(arr['compare_category']=='-'):
            return '-'
        elif (arr['target_category']!='-')&(arr['compare_category']=='-'):
            return '이번주_메인노출'
        elif (arr['target_category']=='-')&(arr['compare_category']!='-'):
            return '지난주_메인노출'
        elif (arr['target_category']!='-')&(arr['compare_category']!='-'):
            return '항상노출'

    def comment_inventory(self, arr):
        oos_amount_threshold = 0 # 재고비교 시 사실상 결품이라고 인정할 정도의 재고 수량이 얼마 이하로 할 것인지
        if (arr['compare_amount']<=oos_amount_threshold)&(arr['target_amount']>arr['compare_amount']):
            return '재입고'
        elif (arr['compare_amount']>oos_amount_threshold)&(arr['target_amount']>arr['compare_amount']):
            return '재고보충'
        elif (arr['target_amount']<=oos_amount_threshold)&(arr['target_amount']<arr['compare_amount']):
            return '품절'
        elif (arr['target_amount']>oos_amount_threshold)&(arr['target_amount']<arr['compare_amount']):
            return '재고감소'
        else:
            return '-'

    def make_exposure_cause(self, result_df):
        # main exposure table
        target_expose = self.mainexposure[(self.mainexposure['date']>self.target_compare_days[2])&(self.mainexposure['date']<=self.target_compare_days[3])]
        target_expose = target_expose.sort_values(by='date', ascending=False).drop_duplicates(subset='product_id')
        target_expose = target_expose.rename(columns={'main_rec_category': 'target_category'})
        compare_expose = self.mainexposure[(self.mainexposure['date']>self.target_compare_days[0])&(self.mainexposure['date']<=self.target_compare_days[1])]
        compare_expose = compare_expose.sort_values(by='date', ascending=False).drop_duplicates(subset='product_id')
        compare_expose = compare_expose.rename(columns={'main_rec_category': 'compare_category'})
        prod_expose = pd.merge(target_expose[['product_id', 'target_category']], compare_expose[['product_id', 'compare_category']], on='product_id', how='outer')

        result_df = pd.merge(result_df, prod_expose, on='product_id', how='left')
        result_df[['target_category', 'compare_category']] = result_df[['target_category', 'compare_category']].fillna('-')
        result_df['exposure_comment']=result_df[['target_category', 'compare_category']].apply(lambda x: self.comment_exposure(x), axis=1)
        return result_df

    def make_inventory_cause(self, result_df):
        # main exposure table
        target_inventory = self.inventory[(self.inventory['sys_time']>self.target_compare_days[2])&(self.inventory['sys_time']<=self.target_compare_days[3])]
        compare_inventory = self.inventory[(self.inventory['sys_time']>self.target_compare_days[0])&(self.inventory['sys_time']<=self.target_compare_days[1])]
        # 재고는 일단위 갱신정보이기 때문에, 합산이 불가하고 특정 날짜 기준으로 보아야 하는데, 현재고가 중요하니 타겟기간은 현재고, 비교기간은 최대한 보수적인 기준으로 max 재고였던날 기준으로 확인
        target_inventory = target_inventory[target_inventory['sys_time']==target_inventory['sys_time'].max()] # 타겟기간은 현재고
        target_inventory = target_inventory.groupby('barcode')['amount'].max().reset_index()
        target_inventory = target_inventory.rename(columns={'amount': 'target_amount'})
        compare_inventory = compare_inventory.groupby('barcode')['amount'].max().reset_index()
        compare_inventory = compare_inventory.rename(columns={'amount': 'compare_amount'})
        bar_inventory = pd.merge(target_inventory, compare_inventory, on='barcode', how='outer')
        bar_inventory = bar_inventory.fillna(0)
        bar_inventory['amt_gap'] = bar_inventory['target_amount'] - bar_inventory['compare_amount']

        result_df = pd.merge(result_df, bar_inventory, on='barcode', how='left')
        result_df['inventory_comment']=result_df[['target_amount', 'compare_amount']].apply(lambda x: self.comment_inventory(x), axis=1)
        return result_df

    def split_top_bot(self, total_df): # 전체 집계 결과를 급상승, 하락으로 분리하는 함수
        target_qty_threshold = 5 # 타겟기간 평균판매량의 threshold. 급상승에서 타겟기간 최소판매기준
        compare_qty_threshold = 5 # 비교기간 평균판매량의 threshold. 급하락에서 과거기간 최소판매기준
        head_num = 20

        total_df = total_df
        # 최종결과
        top_df = total_df[(total_df['target_qty']>=target_qty_threshold) & (total_df['krw_sales_diff']>0)]\
            .sort_values(by=['krw_sales_diff', 'qty_diff', 'growth_rate_daily'], ascending=[False, False, False]).head(head_num) # 성장률 우선정렬, 다음은 qty 기준
        bot_df = total_df[(total_df['compare_qty']>=compare_qty_threshold) & (total_df['krw_sales_diff']<=0)]\
            .sort_values(by=['krw_sales_diff', 'qty_diff', 'growth_rate_daily'], ascending=[True, True, True]).head(head_num)
        return top_df, bot_df

    def prod_result(self, result_df):
        top_df, bot_df = self.split_top_bot(result_df)
        final_result = []
        for temp_df in [top_df, bot_df]:
            prod_detail = temp_df.copy()
            prod_detail = self.make_exposure_cause(prod_detail)
            prod_detail = self.make_inventory_cause(prod_detail)
            prod_detail['rank'] = range(1, len(prod_detail)+1)
            prod_detail = prod_detail[[
                'rank', 'product_name_kor', 'variant_1_name_kor', 'brand', 'krw_sales_diff', 'qty_diff', 'growth_rate_daily', 'target_qty', 'compare_qty', 'target_amount', 'compare_amount', 'inventory_comment', 'exposure_comment', 'amt_gap', 'target_category', 'compare_category', 'target_days', 'compare_days'
            ]] # 순서 변경
            prod_detail = prod_detail.rename(columns=self.col_name_dict)
            prod_detail = prod_detail.fillna('-')

            # 집계결과 필터링
            temp_df['rank'] = range(1, len(temp_df)+1)
            temp_df = temp_df[['rank', 'product_name_kor', 'variant_1_name_kor', 'brand', 'krw_sales_diff', 'qty_diff', 'growth_rate_daily']].reset_index(drop=True)
            temp_df = temp_df.rename(columns=self.col_name_dict)
            final_result.extend([temp_df, prod_detail])
        return final_result

    # def brand_result(self, result_df, prod_df): # @@@ 상품별 급상승/하락 정보를 담고있는 prod_df는 필수 정보. 따라서, 상품별 급상승하락 연산 로직 또한 main.py가 아니라 이 class안에 넣는게 좋을 듯
    #     top_df, bot_df = self.split_top_bot(result_df)
    #     final_result = []
    #     for top_or_bot, temp_df in enumerate([top_df, bot_df]):
    #         brand_detail = []
    #         for brand_target in temp_df['brand'].tolist():
    #             brand_prod = prod_df[prod_df['brand']==brand_target]
    #             if top_or_bot == 0:
    #                 brand_prod = brand_prod.sort_values(by=['krw_sales_diff', 'qty_diff', 'growth_rate_daily'], ascending=[False, False, False])
    #             elif top_or_bot == 1:
    #                 brand_prod = brand_prod.sort_values(by=['krw_sales_diff', 'qty_diff', 'growth_rate_daily'], ascending=[True, True, True])
    #             brand_detail.append(brand_prod)
    #         brand_detail = pd.concat(brand_detail, ignore_index=True)
    #         brand_detail = self.make_exposure_cause(brand_detail)
    #         brand_detail = self.make_inventory_cause(brand_detail)
    #         brand_detail['rank'] = range(1, len(brand_detail)+1)
    #         brand_detail = brand_detail[[
    #             'rank', 'brand', 'product_name_kor', 'variant_1_name_kor', 'krw_sales_diff', 'qty_diff', 'growth_rate_daily', 'target_qty', 'compare_qty', 'target_amount',
    #             'compare_amount','inventory_comment', 'exposure_comment', 'amt_gap', 'target_category', 'compare_category', 'target_days', 'compare_days'
    #         ]] # 순서 변경
    #         brand_detail = brand_detail.rename(columns=self.col_name_dict)
    #         brand_detail = brand_detail.fillna('-')
    #
    #         # 집계결과 필터링
    #         temp_df['rank'] = range(1, len(temp_df)+1)
    #         temp_df = temp_df[['rank', 'brand', 'krw_sales_diff', 'qty_diff', 'growth_rate_daily']].reset_index(drop=True)
    #         temp_df = temp_df.rename(columns=self.col_name_dict)
    #         final_result.extend([temp_df, brand_detail])
    #     return final_result

    def get_agg_detail(self, result_df, prod_df, agg_col): # agg_col
        """
        
        Args:
            result_df: 집계를 원하는 각 테이블
            prod_df: 상품별 결과 테이블 (*agg_col 정보가 포함되어 있어야 함. 없다면 본 함수를 사용 전에 merge를 통해 정보 붙여주어야 함)
            agg_col: 어떤 컬럼 기준으로 집계 및 디테일 테이블을 산출할 것인지(ex. 'brand')

        Returns: top_df, top_df_detail, bot_df, bot_df_detail

        """
        top_df, bot_df = self.split_top_bot(result_df)
        final_result = []
        for top_or_bot, agg_df in enumerate([top_df, bot_df]):
            detail_df = []
            for targets in agg_df[agg_col].tolist():
                target_prod = prod_df[prod_df[agg_col]==targets]
                if top_or_bot == 0:
                    target_prod = target_prod.sort_values(by=['krw_sales_diff', 'qty_diff', 'growth_rate_daily'], ascending=[False, False, False])
                elif top_or_bot == 1:
                    target_prod = target_prod.sort_values(by=['krw_sales_diff', 'qty_diff', 'growth_rate_daily'], ascending=[True, True, True])
                detail_df.append(target_prod)
            detail_df = pd.concat(detail_df, ignore_index=True)
            detail_df = self.make_exposure_cause(detail_df)
            detail_df = self.make_inventory_cause(detail_df)
            detail_df['rank'] = range(1, len(detail_df)+1)
            if agg_col == 'brand':
                detail_df = detail_df[[
                    'rank', agg_col, 'product_name_kor', 'variant_1_name_kor', 'krw_sales_diff', 'qty_diff', 'growth_rate_daily', 'target_qty', 'target_amount',
                    'compare_amount','inventory_comment', 'exposure_comment', 'amt_gap', 'target_category', 'compare_category', 'target_days', 'compare_days'
                ]] # 'brand'가 agg_col일 경우, 필수 노출정보인 브랜드 정보를 중복 노출할 필요가 없음
            else:
                detail_df = detail_df[[
                    'rank', agg_col, 'product_name_kor', 'variant_1_name_kor', 'brand', 'krw_sales_diff', 'qty_diff', 'growth_rate_daily', 'target_qty', 'target_amount', 'compare_amount','inventory_comment', 'exposure_comment', 'amt_gap', 'target_category', 'compare_category', 'target_days', 'compare_days'
                ]] # 'agg_col이 브랜드가 아니면, 필수 노출정보인 브랜드 컬럼을 추가로 포함
            detail_df = detail_df.rename(columns=self.col_name_dict)
            detail_df = detail_df.fillna('-')

            # 집계결과 필터링
            agg_df['rank'] = range(1, len(agg_df)+1)
            agg_df = agg_df[['rank', agg_col, 'krw_sales_diff', 'qty_diff', 'growth_rate_daily']].reset_index(drop=True)
            agg_df = agg_df.rename(columns=self.col_name_dict)
            final_result.extend([agg_df, detail_df])
        return final_result
    
    # def cate_result(self, result_df, prod_df, category_df): # @@@ 상품별 급상승/하락 정보를 담고있는 prod_df는 필수 정보. 따라서, 상품별 급상승하락 연산 로직 또한 main.py가 아니라 이 class안에 넣는게 좋을 듯
    #     prod_df = pd.merge(prod_df, category, on='product_id', how='left') # 카테고리 정보를 미리 prod_sales에 merge할 수 없는 이유가, 카테고리는 1:N 매칭이기 때문. 따라서 미리 merge하면 top을 뽑을 때 문제 발생. cause 파악 단계에서만 merge할 것
    #     cate_detail = []
    #     for cate_target in result_df['category'].tolist():
    #         cate_prod = prod_df[prod_df['category']==cate_target]
    #         cate_prod = cate_prod.sort_values(by='growth_rate_daily', ascending=False)
    #         cate_detail.append(cate_prod)
    #     cate_detail = pd.concat(cate_detail, ignore_index=True)
    #     cate_detail = self.make_exposure_cause(cate_detail)
    #     cate_detail = self.make_inventory_cause(cate_detail)
    #     cate_detail = cate_detail[[
    #         'category', 'product_name_kor', 'variant_1_name_kor', 'brand', 'growth_rate_daily', 'target_qty', 'compare_qty', 'target_amount', 'compare_amount',
    #         'inventory_comment', 'exposure_comment', 'amt_gap', 'target_category', 'compare_category', 'target_days', 'compare_days', 'product_id', 'barcode'
    #     ]] # 순서 변경
    #     cate_detail = cate_detail.rename(columns=self.col_name_dict)
    #     cate_detail.index = cate_detail.index + 1
    #     cate_detail = cate_detail.fillna('-')
    #     return cate_detail
    # 
    # def vendor_result(self, result_df, prod_df, vendor_df): # @@@ 상품별 급상승/하락 정보를 담고있는 prod_df는 필수 정보. 따라서, 상품별 급상승하락 연산 로직 또한 main.py가 아니라 이 class안에 넣는게 좋을 듯
    #     prod_df = pd.merge(prod_df, vendor_df, on='product_id', how='left')
    #     vendor_detail = []
    #     for vendor_target in result_df['company_name'].tolist():
    #         vendor_prod = prod_df[prod_df['company_name']==vendor_target]
    #         vendor_prod = vendor_prod.sort_values(by='growth_rate_daily', ascending=False)
    #         vendor_detail.append(vendor_prod)
    #     vendor_detail = pd.concat(vendor_detail, ignore_index=True)
    #     vendor_detail = self.make_exposure_cause(vendor_detail)
    #     vendor_detail = self.make_inventory_cause(vendor_detail)
    #     vendor_detail = vendor_detail[[
    #         'company_name', 'product_name_kor', 'variant_1_name_kor', 'brand', 'growth_rate_daily', 'target_qty', 'compare_qty', 'target_amount', 'compare_amount',
    #         'inventory_comment', 'exposure_comment', 'amt_gap', 'target_category', 'compare_category', 'target_days', 'compare_days', 'product_id', 'barcode'
    #     ]] # 순서 변경
    #     vendor_detail = vendor_detail.rename(columns=self.col_name_dict)
    #     vendor_detail.index = vendor_detail.index + 1
    #     vendor_detail = vendor_detail.fillna('-')
    #     return vendor_detail

In [8]:
tracker = CauseTracker(mainexposure, inventory, target_compare_days)
prod_top, prod_top_detail, prod_bot, prod_bot_detail = tracker.prod_result(prod_sales)

## 2. 브랜드별 결과

In [9]:
## 2. 브랜드별 결과
target_qty_threshold = 5 # 비교기간 판매가 없는 상품의 타겟기간 평균판매량의 threshold
compare_qty_threshold = 5 # 비교기간 평균판매량의 threshold

daily_sales = analytics.groupby(['product_id', 'purchased_ymd'])['product_qty', 'krw_sales'].sum().reset_index()
daily_sales = pd.merge(daily_sales, brand, on='product_id', how='left')
daily_sales = daily_sales[daily_sales['brand'].notnull()] # 브랜드별 결과이기 때문에, 브랜드 정보 없는 상품은 제외 (단, 브랜드 쿼리 및 테이블 소스 다시 한번 DBMS상에서 확인해볼 것)

target_sales = daily_sales[(daily_sales['purchased_ymd']>target_start)&(daily_sales['purchased_ymd']<=target_end)]
target_sales = target_sales.groupby('brand').agg({'krw_sales': 'sum', 'product_qty':'sum', 'purchased_ymd': pd.Series.nunique})
target_sales = target_sales.rename(columns={'krw_sales': 'target_krw_sales', 'product_qty':'target_qty', 'purchased_ymd':'target_days'})
compare_sales = daily_sales[(daily_sales['purchased_ymd']>compare_start)&(daily_sales['purchased_ymd']<=compare_end)]
compare_sales = compare_sales.groupby('brand').agg({'krw_sales': 'sum', 'product_qty':'sum', 'purchased_ymd': pd.Series.nunique})
compare_sales = compare_sales.rename(columns={'krw_sales': 'compare_krw_sales', 'product_qty':'compare_qty', 'purchased_ymd':'compare_days'})
brand_sales = pd.merge(target_sales, compare_sales, on='brand', how='outer')
brand_sales = brand_sales.reset_index()

# growth_rate 성장률 계산을 위해서는 판매량과 판매일자를 1로 채워줘야 하는데, 실제 값은 0으로 해야하기 때문에 저장해두고 추후 복구
target_days = brand_sales['target_days'].fillna(0)
compare_qty = brand_sales['compare_qty'].fillna(0)
compare_days = brand_sales['compare_days'].fillna(0)
brand_sales['target_qty'] = brand_sales['target_qty'].fillna(0) # target_qty는 0 그대로여도 상관없음. 항상 최우선 분자에 있기 때문. 나머지는 1로 채우기
brand_sales['target_days'] = brand_sales['target_days'].fillna(1)
brand_sales['compare_qty'] = brand_sales['compare_qty'].fillna(1)
brand_sales['compare_days'] = brand_sales['compare_days'].fillna(1)
# brand_sales['growth_rate_daily'] = (brand_sales['target_qty']/brand_sales['target_days'] - brand_sales['compare_qty']/brand_sales['compare_days']) / (brand_sales['compare_qty']/brand_sales['compare_days']) # target과 compare의 날짜 수에 차이가 있을 수 있기 때문에, 일평균 판매량을 구한 후, 그 차이를 비교기간 대비로 연산
brand_sales['growth_rate_daily'] = (brand_sales['target_qty'] - brand_sales['compare_qty']) / brand_sales['compare_qty'] # 위처럼 일단위로 연산하니, 주간판매수량합산차이와 반대방향으로 연산되는 경우가 있어서 변경
brand_sales['target_days'] = target_days
brand_sales['compare_qty'] = compare_qty
brand_sales['compare_days'] = compare_days

# 매출액/판매량 차이 연산
brand_sales['target_krw_sales'] = brand_sales['target_krw_sales'].fillna(0)
brand_sales['compare_krw_sales'] = brand_sales['compare_krw_sales'].fillna(0)
brand_sales['krw_sales_diff'] = brand_sales['target_krw_sales'] - brand_sales['compare_krw_sales']
brand_sales['qty_diff'] = brand_sales['target_qty'] - brand_sales['compare_qty']

# 후처리
brand_sales['compare_qty'] = brand_sales['compare_qty'].astype('int')
brand_sales['compare_days'] = brand_sales['compare_days'].astype('int')
brand_sales['growth_rate_daily'] = np.round(brand_sales['growth_rate_daily'], 2)

# 후처리
brand_sales['compare_qty'] = brand_sales['compare_qty'].astype('int')
brand_sales['compare_days'] = brand_sales['compare_days'].astype('int')
brand_sales['growth_rate_daily'] = np.round(brand_sales['growth_rate_daily'], 4) * 100
brand_sales['krw_sales_diff'] = np.round(brand_sales['krw_sales_diff']/1000) # 천단위 환산
brand_sales = brand_sales.fillna('-')

In [10]:
brand_top, brand_top_detail, brand_bot, brand_bot_detail = tracker.get_agg_detail(brand_sales, prod_sales, agg_col='brand')

## 3. 카테고리별 결과

In [11]:
## 3. 카테고리별 결과
daily_sales = analytics.groupby(['product_id', 'purchased_ymd'])['product_qty', 'krw_sales'].sum().reset_index()
daily_sales = pd.merge(daily_sales, category, on='product_id', how='left')
daily_sales = daily_sales[daily_sales['category'].notnull()] # 브랜드별 결과이기 때문에, 브랜드 정보 없는 상품은 제외 (단, 브랜드 쿼리 및 테이블 소스 다시 한번 DBMS상에서 확인해볼 것)

target_sales = daily_sales[(daily_sales['purchased_ymd']>target_start)&(daily_sales['purchased_ymd']<=target_end)]
target_sales = target_sales.groupby('category').agg({'krw_sales': 'sum', 'product_qty':'sum', 'purchased_ymd': pd.Series.nunique})
target_sales = target_sales.rename(columns={'krw_sales': 'target_krw_sales', 'product_qty':'target_qty', 'purchased_ymd':'target_days'})
compare_sales = daily_sales[(daily_sales['purchased_ymd']>compare_start)&(daily_sales['purchased_ymd']<=compare_end)]
compare_sales = compare_sales.groupby('category').agg({'krw_sales': 'sum', 'product_qty':'sum', 'purchased_ymd': pd.Series.nunique})
compare_sales = compare_sales.rename(columns={'krw_sales': 'compare_krw_sales', 'product_qty':'compare_qty', 'purchased_ymd':'compare_days'})
cate_sales = pd.merge(target_sales, compare_sales, on='category', how='outer')
cate_sales = cate_sales.reset_index()

# growth_rate 성장률 계산을 위해서는 판매량과 판매일자를 1로 채워줘야 하는데, 실제 값은 0으로 해야하기 때문에 저장해두고 추후 복구
target_days = cate_sales['target_days'].fillna(0)
compare_qty = cate_sales['compare_qty'].fillna(0)
compare_days = cate_sales['compare_days'].fillna(0)
cate_sales['target_qty'] = cate_sales['target_qty'].fillna(0) # target_qty는 0 그대로여도 상관없음. 항상 최우선 분자에 있기 때문. 나머지는 1로 채우기
cate_sales['target_days'] = cate_sales['target_days'].fillna(1)
cate_sales['compare_qty'] = cate_sales['compare_qty'].fillna(1)
cate_sales['compare_days'] = cate_sales['compare_days'].fillna(1)
# cate_sales['growth_rate_daily'] = (cate_sales['target_qty']/cate_sales['target_days'] - cate_sales['compare_qty']/cate_sales['compare_days']) / (cate_sales['compare_qty']/cate_sales['compare_days']) # target과 compare의 날짜 수에 차이가 있을 수 있기 때문에, 일평균 판매량을 구한 후, 그 차이를 비교기간 대비로 연산
cate_sales['growth_rate_daily'] = (cate_sales['target_qty'] - cate_sales['compare_qty']) / cate_sales['compare_qty'] # 위처럼 일단위로 연산하니, 주간판매수량합산차이와 반대방향으로 연산되는 경우가 있어서 변경
cate_sales['target_days'] = target_days
cate_sales['compare_qty'] = compare_qty
cate_sales['compare_days'] = compare_days

# 매출액/판매량 차이 연산
cate_sales['target_krw_sales'] = cate_sales['target_krw_sales'].fillna(0)
cate_sales['compare_krw_sales'] = cate_sales['compare_krw_sales'].fillna(0)
cate_sales['krw_sales_diff'] = cate_sales['target_krw_sales'] - cate_sales['compare_krw_sales']
cate_sales['qty_diff'] = cate_sales['target_qty'] - cate_sales['compare_qty']

# 후처리
cate_sales['compare_qty'] = cate_sales['compare_qty'].astype('int')
cate_sales['compare_days'] = cate_sales['compare_days'].astype('int')
cate_sales['growth_rate_daily'] = np.round(cate_sales['growth_rate_daily'], 2)

# 후처리
cate_sales['compare_qty'] = cate_sales['compare_qty'].astype('int')
cate_sales['compare_days'] = cate_sales['compare_days'].astype('int')
cate_sales['growth_rate_daily'] = np.round(cate_sales['growth_rate_daily'], 4) * 100
cate_sales['krw_sales_diff'] = np.round(cate_sales['krw_sales_diff']/1000) # 천단위 환산
cate_sales = cate_sales.fillna('-')

In [12]:
cate_prods = pd.merge(prod_sales, category, on='product_id', how='left') # 카테고리 정보를 미리 prod_sales에 merge할 수 없는 이유가, 카테고리는 1:N 매칭이기 때문. 따라서 미리 merge하면 top을 뽑을 때 문제 발생. cause 파악을 위해 get_agg_detail 함수에 넣기 전에 merge 된 prod_sales를 넣어줘야 함
cate_top, cate_top_detail, cate_bot, cate_bot_detail = tracker.get_agg_detail(cate_sales, cate_prods, agg_col='category')

## 4. 벤더별 결과

In [13]:
## 4. 벤더별 결과
daily_sales = analytics[analytics['company_name'].notnull()] # 벤더별 결과이기 때문에, 벤더 정보 없는 상품은 제외

target_sales = daily_sales[(daily_sales['purchased_ymd']>target_start)&(daily_sales['purchased_ymd']<=target_end)]
target_sales = target_sales.groupby('company_name').agg({'krw_sales': 'sum', 'product_qty':'sum', 'purchased_ymd': pd.Series.nunique})
target_sales = target_sales.rename(columns={'krw_sales': 'target_krw_sales', 'product_qty':'target_qty', 'purchased_ymd':'target_days'})
compare_sales = daily_sales[(daily_sales['purchased_ymd']>compare_start)&(daily_sales['purchased_ymd']<=compare_end)]
compare_sales = compare_sales.groupby('company_name').agg({'krw_sales': 'sum', 'product_qty':'sum', 'purchased_ymd': pd.Series.nunique})
compare_sales = compare_sales.rename(columns={'krw_sales': 'compare_krw_sales', 'product_qty':'compare_qty', 'purchased_ymd':'compare_days'})
vendor_sales = pd.merge(target_sales, compare_sales, on='company_name', how='outer')
vendor_sales = vendor_sales.reset_index()

# growth_rate 성장률 계산을 위해서는 판매량과 판매일자를 1로 채워줘야 하는데, 실제 값은 0으로 해야하기 때문에 저장해두고 추후 복구
target_days = vendor_sales['target_days'].fillna(0)
compare_qty = vendor_sales['compare_qty'].fillna(0)
compare_days = vendor_sales['compare_days'].fillna(0)
vendor_sales['target_qty'] = vendor_sales['target_qty'].fillna(0) # target_qty는 0 그대로여도 상관없음. 항상 최우선 분자에 있기 때문. 나머지는 1로 채우기
vendor_sales['target_days'] = vendor_sales['target_days'].fillna(1)
vendor_sales['compare_qty'] = vendor_sales['compare_qty'].fillna(1)
vendor_sales['compare_days'] = vendor_sales['compare_days'].fillna(1)
# vendor_sales['growth_rate_daily'] = (vendor_sales['target_qty']/vendor_sales['target_days'] - vendor_sales['compare_qty']/vendor_sales['compare_days']) / (vendor_sales['compare_qty']/vendor_sales['compare_days']) # target과 compare의 날짜 수에 차이가 있을 수 있기 때문에, 일평균 판매량을 구한 후, 그 차이를 비교기간 대비로 연산
vendor_sales['growth_rate_daily'] = (vendor_sales['target_qty'] - vendor_sales['compare_qty']) / vendor_sales['compare_qty'] # 위처럼 일단위로 연산하니, 주간판매수량합산차이와 반대방향으로 연산되는 경우가 있어서 변경
vendor_sales['target_days'] = target_days
vendor_sales['compare_qty'] = compare_qty
vendor_sales['compare_days'] = compare_days

# 매출액/판매량 차이 연산
vendor_sales['target_krw_sales'] = vendor_sales['target_krw_sales'].fillna(0)
vendor_sales['compare_krw_sales'] = vendor_sales['compare_krw_sales'].fillna(0)
vendor_sales['krw_sales_diff'] = vendor_sales['target_krw_sales'] - vendor_sales['compare_krw_sales']
vendor_sales['qty_diff'] = vendor_sales['target_qty'] - vendor_sales['compare_qty']

# 후처리
vendor_sales['compare_qty'] = vendor_sales['compare_qty'].astype('int')
vendor_sales['compare_days'] = vendor_sales['compare_days'].astype('int')
vendor_sales['growth_rate_daily'] = np.round(vendor_sales['growth_rate_daily'], 2)

# 후처리
vendor_sales['compare_qty'] = vendor_sales['compare_qty'].astype('int')
vendor_sales['compare_days'] = vendor_sales['compare_days'].astype('int')
vendor_sales['growth_rate_daily'] = np.round(vendor_sales['growth_rate_daily'], 4) * 100
vendor_sales['krw_sales_diff'] = np.round(vendor_sales['krw_sales_diff']/1000) # 천단위 환산
vendor_sales = vendor_sales.fillna('-')

In [14]:
vendor_prods = pd.merge(prod_sales, prod_vendor, on='product_id', how='left') # 카테고리 정보를 미리 prod_sales에 merge할 수 없는 이유가, 카테고리는 1:N 매칭이기 때문. 따라서 미리 merge하면 top을 뽑을 때 문제 발생. cause 파악을 위해 get_agg_detail 함수에 넣기 전에 merge 된 prod_sales를 넣어줘야 함
vendor_top, vendor_top_detail, vendor_bot, vendor_bot_detail = tracker.get_agg_detail(vendor_sales, vendor_prods, agg_col='company_name')

In [15]:
# aggregation file save
today = datetime.today().strftime('%Y%m%d')
path = '{}_급상승하락_집계결과.xlsx'.format(today)
writer = pd.ExcelWriter(path, engine='xlsxwriter')

prod_top.to_excel(writer, sheet_name='상품급상승_집계결과')
prod_bot.to_excel(writer, sheet_name='상품급하락_집계결과')
brand_top.to_excel(writer, sheet_name='브랜드급상승_집계결과')
brand_bot.to_excel(writer, sheet_name='브랜드급하락_집계결과')
cate_top.to_excel(writer, sheet_name='카테고리급상승_집계결과')
cate_bot.to_excel(writer, sheet_name='카테고리급하락_집계결과')
vendor_top.to_excel(writer, sheet_name='벤더급상승_집계결과')
vendor_bot.to_excel(writer, sheet_name='벤더급하락_집계결과')

writer.save()
writer.close()

In [16]:
# detail file save
today = datetime.today().strftime('%Y%m%d')
path = '{}_급상승하락_상품상세.xlsx'.format(today)
writer = pd.ExcelWriter(path, engine='xlsxwriter')

prod_top_detail .to_excel(writer, sheet_name='상품급상승_상품상세')
prod_bot_detail .to_excel(writer, sheet_name='상품급하락_상품상세')
brand_top_detail .to_excel(writer, sheet_name='브랜드급상승_상품상세')
brand_bot_detail .to_excel(writer, sheet_name='브랜드급하락_상품상세')
cate_top_detail .to_excel(writer, sheet_name='카테고리급상승_상품상세')
cate_bot_detail .to_excel(writer, sheet_name='카테고리급하락_상품상세')
vendor_top_detail .to_excel(writer, sheet_name='벤더급상승_상품상세')
vendor_bot_detail .to_excel(writer, sheet_name='벤더급하락_상품상세')

writer.save()
writer.close()